<img src="img/uproot-logo-300px.png" alt="Uproot" width="300px" style="margin-bottom: -50px; margin-right: 20px"><font size="5"> is a pure-Python implementation of ROOT I/O.</font>

<br><br>

<img src="img/abstraction-layers.png" width="900px">


# What will this tutorial use?

New versions of both: <b>Uproot 4</b> and <b>Awkward 1</b>. The code is adapted from Jim Pivarski's PyHEP [talk](https://github.com/jpivarski-talks/2020-07-13-pyhep2020-tutorial).

The required packages are available on PyPI (pie-pee-eye) and can be easily installed with pip.

In [ ]:
import uproot4
import awkward1 as ak
import numpy as np
from matplotlib import pyplot as plt

Loading files is very easy! We will look at some muon data for now

In [ ]:
root = uproot4.open('data/opendata_muons.root')

In [ ]:
root

When you open a file, you get its root directory, which has the properties of a Python dict.

You can list its keys.

In [ ]:
root.keys()

(The `;1` wasn't necesssary—it's a "cycle number," which ROOT uses to distinguish objects in the same directory with the same name. If unspecified, you get the highest cycle number.)

In [ ]:
root.classnames()

# Exploring a TTree

In [ ]:
tree = root["Events"]
tree.show()

These are all the branches of the TTree with the type name of the branch (if Uproot can determine it) and its interpretation as an array (if possible).

TTrees also have a dict-like interface.

In [ ]:
tree.keys()

In [ ]:
tree.items()

In [ ]:
tree.typenames()

We can also filter the keys on name or type, wildcards are allowed!

In [ ]:
tree.keys(filter_name="P*")

In [ ]:
tree.keys(filter_typename="float[]")

# Turning branches into arrays

If a branch has a known interpretation, you can call `array` on it to get an array.

In [ ]:
tree["Muon_pt"].array()

First thing to notice: this is not a NumPy array. It's because the data have different numbers of values in each element (it's a jagged array). It's an array of arrays with possibly different lengths.

In [ ]:
tree["Muon_pt"].array()[:20].tolist()

We can (in Uproot 4) _force_ it to be a NumPy array, but it isn't pretty:

In [ ]:
tree["Muon_pt"].array(library="np")[:20]

The data type (`dtype`) of this NumPy array is `object`, meaning that each element it contains is a Python object, namely another NumPy array.

The default is for all arrays to be Awkward arrays, but you can override this by specifying `library`.

The difference is that Awkward arrays interpret nested lists as a second dimension, whereas NumPy object arrays do not:

In [ ]:
awkward_array = tree["Muon_pt"].array(library="ak")
numpy_array = tree["Muon_pt"].array(library="np")

In [ ]:
# from the first 20 events, get the first item
awkward_array[:20, 0]

In [ ]:
# doesn't work with NumPy object arrays because contents are not guaranteed to be arrays
numpy_array[:20, 0]

Another valid library is Pandas. Pandas has its own way of describing variable length structures (`MultiIndex`).

In [ ]:
tree["Muon_pt"].array(library="pd")

# Pluralization

If you look carefully, you'll notice that there's an `array` function and an `arrays` function. The latter gets multiple arrays. In the below example we get that as NumPy arrays, however `library="ak"` and `library="pd"` are also valid options to get Awkward arrays and a pandas dataframe respectively.

In [ ]:
# NumPy arrays in a dict
pv_numpy = tree.arrays(filter_name="PV_*", library="np")
pv_numpy

In [ ]:
pv_numpy["PV_x"]

Above, we used `filter_name` to select branches that match a pattern. We can also request specific branches:

In [ ]:
tree.arrays(["PV_x", "PV_y", "PV_z"])

Or do calculations. (This feature exists for TTree aliases, which can be formulas.) We can even apply cuts!

In [ ]:
tree.arrays("PV", aliases={"PV": "sqrt(PV_x**2 + PV_y**2 + PV_z**2)"})

In [ ]:
tree.arrays("PV", cut="sqrt(PV_x**2 + PV_y**2) < 0.1", aliases={"PV": "sqrt(PV_x**2 + PV_y**2 + PV_z**2)"})

# Multiple files

So far so good, but this was just one small file. Taking a look at `/data/bfys`:

<img src="img/data_bfys.png" width="200px">

What if I have 100+ GBs of ROOT files that I want to open? The Nikhef laptop budget only allowed me to buy a laptop with 16GB RAM, help me!!

## Concatenation

The simplest way to deal with opening multiple files is to read a selection of branches entirely into memory, concatenating them. If you have enough memory, and your files are small enough, go for it!

In [ ]:
all_in_memory = uproot4.concatenate("data/uproot-sample-*.root:sample", ["i4", "ai8", "Af8", "str"])
all_in_memory

In [ ]:
all_in_memory.i4

In [ ]:
all_in_memory.ai8

In [ ]:
all_in_memory.Af8

In [ ]:
all_in_memory.str

But, often enough, you don't have enough memory. What then? 

## Laziness

One option is to open them as lazy arrays, which opens the files (to get the number of events in each), but doesn't read the data until you use it.

In [ ]:
not_in_memory_yet = uproot4.lazy("data/uproot-sample-*.root:sample")
not_in_memory_yet

"If it's not in memory, how can I see the values?"

Only the parts of the files (branches and batches of events) that are visible are read. In the above, `n` and `b` from the first file and `str` from the last file must have been read.

Let's get the `Af8` field:

In [ ]:
not_in_memory_yet.Af8

Again, this only read from the first and last files to show the first and last values.

A mathematical operation would cause them all to be read in.

In [ ]:
not_in_memory_yet.Af8 + 100

## Controlling the lazy cache

After (part of) a lazy array has been read, how long does it stay in memory? Is it constantly being re-read every time we do a calculation?

By default, a 100 MB cache is associated with the lazy array, but we can provide our own if we want a bigger or smaller one.

In [ ]:
cache = uproot4.LRUArrayCache("1 GB")
not_in_memory_yet = uproot4.lazy("data/uproot-sample-*.root:sample", array_cache=cache)

In [ ]:
cache

In [ ]:
not_in_memory_yet

In [ ]:
cache

In [ ]:
not_in_memory_yet + 100

In [ ]:
cache

What's more, we can clear it when we need to.

In [ ]:
cache.clear()

In [ ]:
cache

## Iteration

But often, that's still not enough control.

We don't read arrays into memory for the fun of it, we do it to perform calculations, and lazy arrays don't control which parts of which arrays are in memory during a calculation.

If you're worried about memory, the safest thing to do is to iterate over the data.

In [ ]:
for arrays in uproot4.iterate("data/uproot-sample-*.root:sample", ["i4", "Af8"]):
    print(arrays["i4"] + arrays["Af8"])

This iteration is _not_ one event at a time. (This set of TTrees has 420 entries.) It's a _chunk of events_ at a time.

In each step, a chunk of events for all specified arrays (`["i4", "Af8"]`) is read. You do your calculation, move on to the next step, and all the previous arrays are dropped. (Only TBasket data carries over if event steps don't line up with TBasket boundaries—a low-level detail.)

# Writing to ROOT files

Uproot 4 cannot write to ROOT files yet: see Uproot 3 documentation.

Some caveats, though:

   * Writing ROOT files with Uproot will always be minimal: histograms and only basic types in TTrees.
   * You won't be able to update an existing file, only make new files.
   * It won't be as fast as writing with ROOT.

The issues involved in _writing_ an established format are considerably different from _reading_. If anyone thinks they can do better, they're welcome to try!

In [ ]:
tree = uproot4.open("data/opendata_muons.root:Events") # can also instantly open a tree by adding it after the filepath
tree.show()

We've already seen that it's "awkward" to deal with the jagged arrays in NumPy. However, they look and feel like records if "zipped" into an Awkward array.

In [ ]:
events = tree.arrays(library="ak", how="zip")
events

We can use the awkward array data type to encapsulate the structure of the events.

In [ ]:
ak.type(events)

`1000000 *` means that there are a million events, `"Muon": var *` means that the contents of the `"Muon"` field are jagged: there's a variable number of them per event.

We could look at a few of these as Python lists and dicts.

In [ ]:
ak.to_list(events[:3])

But the data are not actually arranged as objects in memory; each field (`"pt"`, `"eta"`, `"phi"`, etc.) is in an array by itself.

This means that structure-changing things like pulling out the kinematics are not expensive computations. (That is, they do not scale with the size of the dataset.)


In [ ]:
events["Muon", ["pt", "eta", "phi"]]

In [ ]:
ak.type(events["Muon", ["pt", "eta", "phi"]])

In [ ]:
events["Muon", "pz"] = events["Muon", "pt"] * np.sinh(events["Muon", "eta"])

In [ ]:
ak.type(events.Muon)

In [ ]:
events.Muon.pz

Since nearly all of you are familiar with NumPy, slicing arrays with boolean arrays is probably familiar to you.

What's new is that the boolean arrays can now be jagged to slice jagged arrays (i.e. cut particles).

In [ ]:
events.Muon.pt > 20

In [ ]:
events.Muon[events.Muon.pt > 20]

To cut on events, we need to make the jagged array of booleans into a one-dimensional array of booleans. You can do this with a reducer (such as [ak.sum](https://awkward-array.readthedocs.io/en/latest/_auto/ak.sum.html) or [ak.max](https://awkward-array.readthedocs.io/en/latest/_auto/ak.max.html), but most likely [ak.any](https://awkward-array.readthedocs.io/en/latest/_auto/ak.any.html) and [ak.all](https://awkward-array.readthedocs.io/en/latest/_auto/ak.all.html) for booleans).

In [ ]:
ak.any(events.Muon.pt > 20, axis=1)

In [ ]:
events.Muon[ak.any(events.Muon.pt > 20, axis=1)]

# Awkward analysis

Several new operations are needed when arrays can have arbitrary data structures, so the Awkward Array library is best seen as a collection of functions acting on [ak.Array](https://awkward-array.readthedocs.io/en/latest/_auto/ak.Array.html) (the array type).

Probably the most important of these is [ak.num](https://awkward-array.readthedocs.io/en/latest/_auto/ak.num.html), which tells us the number of elements in each nested list.

In [ ]:
ak.num(events.Muon)

That's how many muons there are in each event.

This becomes necessary if we ever try to select the first (and second, etc.) element in each event. Some events might not have any, and this will therefore yield an error.

In [ ]:
events.Muon[:, 0]

So we can use [ak.num](https://awkward-array.readthedocs.io/en/latest/_auto/ak.num.html) to slice the first dimension and remove the events with 0 muons.

In [ ]:
events.Muon[ak.num(events.Muon) > 0, 0]

## Masking vs cutting

In the nearly two years that physicists have been doing analyses with Awkward Arrays, they've found that cuts are difficult to accumulate. If the first cut changes the length of the array from 1000000 to 969031, boolean arrays that could be applied to the first array can't be applied to the second array.

In practice, they've taken a logical-and of all cuts and apply them at the end, but we can do better: we can mask, rather than cut.

In [ ]:
events.Muon.mask[events.Muon.pt > 20]

One of the new types that Awkward Array introduces is the "option type," which allows some values to be `None`. (It's a `?` in the type specification.)

## Making regular arrays

If you're feeding your data into a machine learning pipeline, you might need the jaggedness to go away. There are functions for padding jagged arrays (with `None`) so that they reach a desired length (and replacing `None` with a preferred value): [ak.pad_none](https://awkward-array.readthedocs.io/en/latest/_auto/ak.pad_none.html) (and [ak.fill_none](https://awkward-array.readthedocs.io/en/latest/_auto/ak.fill_none.html)).

In [ ]:
ak.pad_none(events.Muon.pt, 3, clip=True)

In [ ]:
ak.fill_none(ak.pad_none(events.Muon.pt, 3, clip=True), 0)

When you're plotting something, you usually want to flatten the jaggedness. To get the muon pT distribution we can do:

In [ ]:
plt.hist(ak.flatten(events.Muon.pt), bins=120, range=(0, 60));

## Awkward combinatorics

We often want to find pairs of particles with some invariant mass. To do that, we need combinatoric functions like [ak.cartesian](https://awkward-array.readthedocs.io/en/latest/_auto/ak.cartesian.html) and [ak.combinations](https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html).

<table style="margin-left: 0px">
    <tr style="background: white"><td style="font-size: 1.75em; font-weight: bold; text-align: center">Cartesian product (per event)</td><td style="font-size: 1.75em; font-weight: bold; text-align: center">n-choose-k combinations (per event)</td></tr>
    <tr style="background: white"><td><img src="img/cartoon-cartesian.png"></td><td><img src="img/cartoon-combinations.png"></td></tr>
</table>

In [ ]:
muon_pairs = ak.combinations(events.Muon, 2)
muon_pairs

In [ ]:
m1, m2 = ak.unzip(muon_pairs)
m1, m2

In [ ]:
masses = np.sqrt(2*m1.pt*m2.pt*(np.cosh(m1.eta - m2.eta) - np.cos(m1.phi - m2.phi)))
masses

In [ ]:
plt.hist(ak.flatten(masses), bins=80, range=(70, 110))

In [ ]:
plt.hist(ak.flatten(masses), bins=240, range=(0, 12))
plt.yscale("log")

# Histograms!

But Lex, this is all very nice, but what about my beautiful ROOT plots? What do I do with those? Can I still use those?

Excellent question!

Uproot can read histograms (as well as most other ROOT objects), but it doesn't deal directly with them. The first thing that you do when extracting a histogram is to convert it to another library.

In [ ]:
histograms = uproot4.open("data/hepdata-example.root")

In [ ]:
histograms.classnames()

In [ ]:
histograms["hpx"].to_boost()

This is a [boost-histogram](https://github.com/scikit-hep/boost-histogram), a clean design of N-dimensional histograms in the [Boost](https://www.boost.org/doc/libs/release/libs/histogram/doc/html/index.html) C++ library (with Python bindings). Boost-histogram focuses just on **filling and manipulating (e.g. slicing)** histograms.

But we want to plot it, right? There's another library, [mplhep](https://github.com/scikit-hep/mplhep), which focuses just on **plotting** histograms in Matplotlib.

<table style="margin-left: 0px">
    <tr style="background: white">
        <td><img src="img/BoostHistogramCppLogo.png" width="300px" style="margin-right: 20px"></td>
        <td><img src="img/BoostHistogramPythonLogo.png" width="300px" style="margin-right: 20px"></td>
        <td><img src="img/mplhep.png" width="300px"></td>
    </tr>
</table>

Now we can create a histogam in one line!

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep

hep.histplot(histograms["hpx"].to_boost())

# mplhep
What is this magic mplhep? mplhep is described as "a set of helpers for matplotlib to more easily produce plots typically needed in HEP as well as style them in way that's compatible with current collaboration requirements (ROOT-like plots for CMS, ATLAS, LHCb, ALICE)." Let's take a look!

Let's start looking at some toy data

In [ ]:
H = np.histogram(np.random.normal(2.5, .5, 100), bins=np.arange(0,6, 0.5))
print("Type:", type(H))
h, bins = H
print("Values:", h)
print("Bins:", bins)

`mplhep.histplot()` for mplhep is what `plt.hist()` is for matplotlib:

#### Primary goal is to stay unobtrusive
- if you know how `plt.hist()` works, `mplhep.histplot()` should behave like you'd expect
- painless transition back if `mpl` grows a proper histogram plotting method
- kwargs you are used to should work

In [ ]:
f, axs = plt.subplots(1,2, figsize=(14, 5))

hep.histplot(H, ax=axs[0])
hep.histplot(h, bins, yerr=True, ax=axs[1]);

In [ ]:
f, axs = plt.subplots(1,2, figsize=(14, 5))

hep.histplot(H, ax=axs[0], histtype='fill', hatch='//')
hep.histplot(H, ax=axs[1], histtype='errorbar', yerr=True, c='black')

Should be easy to use if you're used to matplotlib!

In [ ]:
f, axs = plt.subplots(1,3, figsize=(21, 5))

hep.histplot([h, h*2], bins=bins, ax=axs[0], yerr=True, label=["MC1", "MC2"])
hep.histplot(np.random.poisson(h*3), bins=bins, ax=axs[1], yerr=True, label="Data")
hep.histplot([h, h*2], bins=bins, ax=axs[2], stack=True, label=["MC1", "MC2"], density=True)
hep.histplot(np.random.poisson(h*3), bins=bins, ax=axs[2], yerr=True, histtype='errorbar', label="Data", density=True)
for ax in axs:
    ax.legend()
axs[0].set_title("Some MCs")
axs[1].set_title("Draw Poisson Data")
axs[2].set_title("Data/MC Shape comparison"); 

### Let's load some data and see how that works
Maybe we can make some nice Dalitz plots

In [ ]:
# Load Dalitz data from the LHCb Starterkit
dalitz_data = uproot4.open("data/dalitz2.root:tree").arrays(library='np')
dalitz_data

Let's explore the data, we have the mass squared for the comination of particles A and B and for A and C.

In [ ]:
h_AB, bins = np.histogram(dalitz_data["M2AB"], bins=100)
h_AC, bins_AC = np.histogram(dalitz_data["M2AC"], bins=100)

In [ ]:
f, axs = plt.subplots(1,2, figsize=(15, 7))
hep.histplot(h_AB,bins,ax=axs[0], label="M2AB")
hep.histplot(h_AC,bins,ax=axs[1], label="M2AC")

Now let's make a 2D plot!

In [ ]:
H, xedges, yedges = np.histogram2d(dalitz_data["M2AB"], dalitz_data["M2AC"], bins=256)
hep.hist2dplot(H, xedges, yedges)
plt.gca().invert_yaxis()

## Default plot styles
But this just seems like matplotlib with extra steps? Where is the HEP functionality?

Well,`mplhep` comes with default plot styles! It has ATLAS, CMS, and most interestingly LHCb! It even comes with the LHCb label.

In [ ]:
plt.style.use(hep.style.LHCb)
hep.histplot(h_AB,bins, label="M2AB")
plt.xlabel("M2AB [GeV^2]")
plt.ylabel("Entries")
hep.lhcb.label(data=True, paper=False, year=2020, lumi="420")

In [ ]:
hep.lhcb.label?

In [ ]:
hep.hist2dplot(H, xedges, yedges)
hep.lhcb.label() # default to 2017 simulation=True
plt.xlabel("M2AB [GeV^2]")
plt.ylabel("M2AC [GeV^2]")
plt.gca().invert_yaxis()

We can also make plots look ROOT style, for the ROOT fans under us

In [ ]:
plt.style.use(hep.style.ROOT)
hep.histplot(h_AB,bins, label="M2AB")
plt.xlabel("M2AB [GeV]")
plt.ylabel("Entries")
hep.lhcb.label(data=True, paper=False, year=2020, lumi="420")

In [ ]:
hep.hist2dplot(H, xedges, yedges)
plt.gca().invert_yaxis()


# boost-histogram

Okay, but what if I really like ROOT histograms? NumPy and matplotlib are just now the same, I can't even fill!

The Python ecosystem is missing a good Histogram object. NumPy can perform a histogram operation, but it does not produce an object, and there are limitations and performance issues. The closest thing we have to a histogram is in ROOT.

Let's start with the basics. We will create a histogram using boost-histogram and fill it.

In [ ]:
import boost_histogram as bh

In [ ]:
data1 = np.random.normal(3.5, 2.5, size=1_000_000) # toy data with 1 million entries
hist1 = bh.Histogram(bh.axis.Regular(40, -2, 10))  # create boost histogram object

Like ROOT, we can fill _after_ we make a histogram, as many times as we want. You can fill single values, but to take advantage of the performance, you should fill with arrays.

In [ ]:
hist1.fill(data1)

You can see that the histogram has been filled. Let's explicitly check to see how many entries are in the histogram:

In [ ]:
hist1.sum()

Huh, that's not 1 million!! What happened to the missing items? They are in the underflow and overflow bins:

In [ ]:
hist1.sum(flow=True)

Like ROOT, we have overflow bins by default. We can turn them off, but they enable some powerful things like projections.

Let's plot this:

In [ ]:
plt.bar(hist1.axes[0].centers, hist1.view(), width=hist1.axes[0].widths);

Note: You can leave off the `.view()` if you want to - histograms conform to the buffer protocol. Also, you can select the axes before or after calling `.centers`; this is very useful for ND histograms.

From now on, let's be lazy, and use mplhep, which natively supports boost-histogram now. See, it all ties in together!

In [ ]:
plt.style.use(hep.style.ROOT)
hep.histplot(hist1)

You should think of boost-histogram like NumPy: No plotting is built in, but the data is easy to access.

Unlike ROOT, Histograms are built of of basic building blocks: 1 or more **axes**, and a **storage**. The storage holds **accumulators**, which can be simple doubles or ints, or more complex things that hold extra information about the operation (which might not even be a sum! (generalized histograms).

## Integration with NumPy

To start using this yourself, you don't even need to change your code. Let's try the NumPy adapters.

In [ ]:
bins2, edges2 = bh.numpy.histogram(data1, bins=10) # using boost
b2, e2 = np.histogram(data1, bins=10)              # using NumPy

In [ ]:
bins2 - b2

In [ ]:
e2 - edges2

Not bad! Let's start moving to the boost-histogram API, so we can use the plotting function we just learned about:

In [ ]:
hist2 = bh.numpy.histogram(data1, bins="auto", histogram=bh.Histogram)
hep.histplot(hist2);

Now we can move over to boost-histogram one step at a time! Just to be complete, we can also go back to a Numpy tuple from a Histogram object:

In [ ]:
b2p, e2p = bh.numpy.histogram(data1, bins=10, histogram=bh.Histogram).to_numpy()
b2p == b2

And, while "recently" NumPy optimized 1D regular binning, it still beats optimized NumPy:

In [ ]:
%%timeit
bh.numpy.histogram(data1, bins=100)

In [ ]:
%%timeit
np.histogram(data1, bins=100)

## Multiple dimension
Now we all like multiple dimension histograms, boost also has a solution for that! Let's reuse the Dalitz data for that.

In [ ]:
hist3 = bh.Histogram(bh.axis.Regular(256, 0, 1250,), bh.axis.Regular(256, 0., 1700))

In [ ]:
data2d = [dalitz_data['M2AB'], dalitz_data["M2AC"]]

In [ ]:
hist3.fill(*data2d)

In [ ]:
plt.pcolormesh(*hist3.axes.edges.T, hist3.view().T);
plt.gca().invert_yaxis()

This is transposed because of differing indexing conventions. Let's try 3D!

In [ ]:
data3d = [np.random.normal(size=1_000_000) for _ in range(3)]

hist3d = bh.Histogram(
    bh.axis.Regular(150, -5, 5),
    bh.axis.Regular(100, -5, 5),
    bh.axis.Regular(100, -5, 5),
)

hist3d.fill(*data3d)

Let's project to the first two axes:

In [ ]:
hep.hist2dplot(hist3d[:, :, sum]);

## boost UHI

<img alt="Diagram of UHI" src="img/boost.png"></img> 

Let's explore the boost-histogram UHI syntax. With this we can easily find bin content, get slices, rebin, etc. We will start with a 1D histogram:

In [ ]:
h = bh.Histogram(bh.axis.Regular(100, -3.5, 3.5))
data = np.concatenate([
    np.random.normal(-.75,.3, 1_000_000),
    np.random.normal(.75,.3, 750_000),
    np.random.normal(-1.5,.2, 200_000),
])

h.fill(data)

In [ ]:
hep.histplot(h)

We can see that we want x from -2 to 0, in *data coordinates*:

In [ ]:
hep.histplot(h[bh.loc(-2):bh.loc(0)]);

What's the contents of a bin?

In [ ]:
h[bh.loc(-.75)]

How about reducing a histogram? Let's try the previous 2D Histogram

In [ ]:
hep.histplot(hist3[:, sum])
hep.histplot(hist3[sum, :]);

Let's look at one part and rebin!

In [ ]:
hep.hist2dplot(hist3[200: 800 : bh.rebin(1), 200 :800: bh.rebin(1)]);
plt.gca().invert_yaxis()

How many entries do we have at M2AB = 500 and M2AC = 600?

In [ ]:
hist3[bh.loc(500), bh.loc(600)]

## No time to cover here:
boost has waaaaay more functions for which we don't have the time today. A small selection is listed below:

- Continuous vs discrete axis
- Variable bin width
- Profile histograms
- Density histograms
- A lot more!

More information can be found on the [GitHub page](https://github.com/scikit-hep/boost-histogram)